# Races information

This notebook describes the steps to get a final table describing the parsed races at a high level.

In [1]:
import numpy as np
import pandas as pd

## Getting weather

We start from the file [`links2runs.csv`](../datasets/links2runs.csv), containing all links to the races/competitions we care about (this same file is also used to get *detailed* information about the results and participants in each race).

Using this file and the two scripts [`weather.py`](../weather/weather.py) and [`weather_utils.py`](../weather/weather_utils.py), we can get the weather conditions for half of the races, using the [World Weather Online API](https://developer.worldweatheronline.com/api/). The result is the file [`races-information-weather.csv`](../datasets/races-information-weather.csv).

In [2]:
races_weather = pd.read_csv('../datasets/races-information-weather.csv', index_col=0)
races_weather.tail()

,Unnamed: 0.1,Date,Name,Place,URL,min_temp,max_temp,uv_index,weather_desc
2003,2003,sam. 12.12.2015,"Christmas Midnight Run, Lausanne",Lausanne,http://services.datasport.com/2015/lauf/midnight,6.0,15.0,0.0,Clear
2004,2004,sam. 12.12.2015,"La Trotteuse-Tissot, La Chaux-de-Fonds",La Chaux-de-Fonds,http://services.datasport.com/2015/lauf/trotteuse,2.0,8.0,0.0,Clear
2005,2005,sam. 05.12.2015,"Course de l'Escalade, Genève",Genève,http://services.datasport.com/2015/lauf/escalade,0.0,9.0,0.0,Sunny
2006,2006,sam. 05.12.2015,Gossauer Weihnachtslauf,Gossau SG,http://services.datasport.com/2015/lauf/gossau,4.0,14.0,0.0,Clear
2007,2007,sam. 05.12.2015,Course à Travers Aigle,Aigle,http://services.datasport.com/2015/lauf/aigle,NaN,NaN,NaN,NaN


Despite some effort on our weather scripts, we added a dummy column as a side effect, let's remove it.

In [3]:
races_weather_clean = races_weather.drop('Unnamed: 0.1', axis=1).drop_duplicates().reset_index(drop=True)
races_weather_clean.columns = ['date','name','location','url','min_temp','max_temp','uv_index','weather_desc']
print(races_weather_clean.shape)
races_weather_clean.tail()

(2004, 8)


,date,name,location,url,min_temp,max_temp,uv_index,weather_desc
1999,sam. 12.12.2015,"Christmas Midnight Run, Lausanne",Lausanne,http://services.datasport.com/2015/lauf/midnight,6.0,15.0,0.0,Clear
2000,sam. 12.12.2015,"La Trotteuse-Tissot, La Chaux-de-Fonds",La Chaux-de-Fonds,http://services.datasport.com/2015/lauf/trotteuse,2.0,8.0,0.0,Clear
2001,sam. 05.12.2015,"Course de l'Escalade, Genève",Genève,http://services.datasport.com/2015/lauf/escalade,0.0,9.0,0.0,Sunny
2002,sam. 05.12.2015,Gossauer Weihnachtslauf,Gossau SG,http://services.datasport.com/2015/lauf/gossau,4.0,14.0,0.0,Clear
2003,sam. 05.12.2015,Course à Travers Aigle,Aigle,http://services.datasport.com/2015/lauf/aigle,NaN,NaN,NaN,NaN


## Getting coordinates

At this point we can also add coordinates that were retrieved in [`geography.csv`](../datasets/geography.csv).

In [5]:
geography = pd.read_csv('../datasets/geography.csv', encoding='latin1', index_col=0)
geography.columns = ['location','latitude','longitude']
print(geography.shape)
geography.head()

(11247, 3)


,location,latitude,longitude
0,St-Légier-La Chiésaz,46.47171849999999,6.876770899999999
1,St-Légier,46.47171849999999,6.876770899999999
3,Ernen,46.39850010000001,8.1457733
4,Lausanne,46.5196535,6.6322734
5,Pully,46.5092681,6.6654949


We merge the dataframes to build a more complete one :

In [6]:
races_gps = pd.merge(races_weather_clean, geography, on='location', how='left').drop_duplicates()
races_gps = races_gps.reset_index(drop=True)
print(races_gps.shape)
races_gps.tail()

(2004, 10)


,date,name,location,url,min_temp,max_temp,uv_index,weather_desc,latitude,longitude
1999,sam. 12.12.2015,"Christmas Midnight Run, Lausanne",Lausanne,http://services.datasport.com/2015/lauf/midnight,6.0,15.0,0.0,Clear,46.5196535,6.6322734
2000,sam. 12.12.2015,"La Trotteuse-Tissot, La Chaux-de-Fonds",La Chaux-de-Fonds,http://services.datasport.com/2015/lauf/trotteuse,2.0,8.0,0.0,Clear,47.1034892,6.832783800000001
2001,sam. 05.12.2015,"Course de l'Escalade, Genève",Genève,http://services.datasport.com/2015/lauf/escalade,0.0,9.0,0.0,Sunny,46.2043907,6.1431577
2002,sam. 05.12.2015,Gossauer Weihnachtslauf,Gossau SG,http://services.datasport.com/2015/lauf/gossau,4.0,14.0,0.0,Clear,47.4155608,9.248852
2003,sam. 05.12.2015,Course à Travers Aigle,Aigle,http://services.datasport.com/2015/lauf/aigle,NaN,NaN,NaN,NaN,46.3190253,6.970566


## Parsing dates

Final step : we can parse the dates, in case it's useful later..

In [7]:
races_final = races_gps.copy()

In [8]:
week_dict = {
    'lun': 'monday',
    'mar': 'tuesday',
    'mer': 'wednesday',
    'jeu': 'thursday',
    'ven': 'friday',
    'sam': 'saturday',
    'dim': 'sunday'
}

In [9]:
races_final['weekday'] = races_final.date.apply(lambda x: week_dict[x.split('.')[0].strip()])
races_final['day'] = races_final.date.apply(lambda x: int(x.split('.')[1].strip()))
races_final['month'] = races_final.date.apply(lambda x: int(x.split('.')[2].strip()))
races_final['year'] = races_final.date.apply(lambda x: int(x.split('.')[3].strip()))

In [10]:
races_final.head()

,date,name,location,url,min_temp,max_temp,uv_index,weather_desc,latitude,longitude,weekday,day,month,year
0,sam. 27.03.1999,Männedörfler Waldlauf,Männedorf,http://services.datasport.com/1999/zkb/maennedorf,NaN,NaN,NaN,NaN,47.2574625,8.6946733,saturday,27,3,1999
1,sam. 20.03.1999,Kerzerslauf,Kerzers,http://services.datasport.com/1999/lauf/kerzers,NaN,NaN,NaN,NaN,46.97488999999999,7.1954365,saturday,20,3,1999
2,sam. 24.04.1999,Luzerner Stadtlauf,Luzern,http://services.datasport.com/1999/lauf/luzern,NaN,NaN,NaN,NaN,47.05016819999999,8.3093072,saturday,24,4,1999
3,sam. 24.04.1999,20km de Lausanne,Lausanne,http://services.datasport.com/1999/lauf/km20,NaN,NaN,NaN,NaN,46.5196535,6.6322734,saturday,24,4,1999
4,sam. 24.04.1999,"Chäsitzerlouf, Kehrsatz",Kehrsatz,http://services.datasport.com/1999/lauf/kehrsatz,NaN,NaN,NaN,NaN,NaN,NaN,saturday,24,4,1999


## Exporting

In [11]:
races_final.to_csv('../datasets/races-information.csv')